In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_3월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(802683, 2) Index(['split_str', 'org_idx'], dtype='object')
802683


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 802683/802683 [14:35:41<00:00, 15.28it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['전시회', '추천', '리움', '미술관', '회전', '인간', '일곱', '개', '질문', '전남']
['하다', '가다', '싶다', '하다', '보이다', '오다', '오다', '보다', '생기다', '되다']
['많다', '불가하다', '뜨겁다', '반갑다', '편하다', '비슷하다', '많다', '깔끔하다', '이쁘다', '있다']
['의', '의', '이', '을', '한', '에요', '가', '이', '까지', '을']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

142220


,명사,명사빈도
0,것,192445
1,수,110486
2,그,101803
3,이,90153
4,사람,63553
...,...,...
142215,롬바르디아어,1
142216,콰토,1
142217,키아브레,1
142218,베롤트,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1820


,동사,동사빈도
0,하다,788966
1,되다,164652
2,보다,83887
3,않다,73719
4,되어다,69817
...,...,...
1815,어기적거리다,1
1816,너덜대다,1
1817,간종이다,1
1818,오므라들다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1842


,형용사,형용사빈도
0,있다,357852
1,없다,67837
2,이다,57802
3,좋다,54204
4,같다,53963
...,...,...
1837,곱슬곱슬하다,1
1838,엉큼하다,1
1839,휘영하다,1
1840,방분하다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/3월/나무문화_3월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

398


,조사,조사빈도
0,을,653783
1,의,534492
2,이,481412
3,에,451557
4,를,316231
...,...,...
393,에게다,1
394,만으로써,1
395,까지로서,1
396,까지는커녕,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_3월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,4월 전시회 추천 리움 미술관 순회전 인간 일곱 개의 질문 ft 전남도립미술관 인간...,0,"[4월, 전시회, 추천, 리움, 미술관, 순, 회전, 인간, 일곱, 개, 의, 질문...","[(4월, Number), (전시회, Noun), (추천, Noun), (리움, N..."
1,그때 너무 가고 싶었는데 어찌나 인기가 많던지 예약이 불가했어요,0,"[그때, 너무, 가고, 싶었는데, 어찌나, 인기, 가, 많던지, 예약, 이, 불가했어요]","[(그때, Noun), (너무, Adverb), (가다, Verb), (싶다, Ve..."
2,전시 마지막날까지 전일매진 을 기록하는 뜨거운 반응을 보였는데요 그 전시가 이곳 전...,0,"[전시, 마지막, 날, 까지, 전일, 매진, 을, 기록, 하는, 뜨거운, 반응, 을...","[(전시, Noun), (마지막, Noun), (날, Noun), (까지, Josa..."
3,그 전시를 오늘은 일을 핑계로 이렇게 편히 와서 봤네요,0,"[그, 전시, 를, 오늘, 은, 일, 을, 핑계, 로, 이렇게, 편히, 와서, 봤네요]","[(그, Noun), (전시, Noun), (를, Josa), (오늘, Noun),..."
4,전시명 인간 일곱 개의 질문 장소 전남도립미술관 주소 전남 광양시 광양읍 순광로 6...,0,"[전시, 명, 인간, 일곱, 개, 의, 질문, 장소, 전남, 도립, 미술관, 주소,...","[(전시, Noun), (명, Suffix), (인간, Noun), (일곱, Nou..."
...,...,...,...,...
802678,엑실레스 요새 Forte di Exilles 카살레 몬페라토 또한 인상적인 방어 체...,16127,"[엑실레스, 요새, Forte, di, Exilles, 카, 살레, 몬페라토, 또한...","[(엑실레스, Noun), (요새, Noun), (Forte, Alpha), (di..."
802679,피네롤로 또한 요새화된 중요한 성채로 지리적 중요성으로 인하여 17세기 동안 프랑스...,16127,"[피, 네롤, 로, 또한, 요새화, 된, 중요한, 성채, 로, 지리, 적, 중요성,...","[(피, Noun), (네롤, Noun), (로, Josa), (또한, Noun),..."
802680,베루아 사보이아 Verrua Savoia 의 요새는 복잡한 성벽을 갖추고 있어 17...,16127,"[베, 루아, 사보이아, Verrua, Savoia, 의, 요새, 는, 복잡한, 성...","[(베, Noun), (루아, Noun), (사보이아, Noun), (Verrua,..."
802681,니차 또한 바다로 나가는 유일한 출구라는 중요성으로 인하여 요새화되었다,16127,"[니, 차, 또한, 바다로, 나가는, 유일한, 출구, 라는, 중요성, 으로, 인하여...","[(니, Modifier), (차, Noun), (또한, Noun), (바다로, N..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_3월_형태소분석.txt", index=False, encoding='UTF-8')